In [1]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
# from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
# from paquetes.utils import *
import pandas as pd

# Configuramos rutas
DRIVER_DIR = "C:\\chromedriver_win32\\chromedriver.exe"
OUTPUT_DIR = "C:\\Users\\Matias\\Documents\\Matias\\Repositorios\\Web Scrapping\\Diarios\\output"

In [2]:
# Funciones que ayudan a pasar a un formato timestamp en la fecha de la noticia.
def mes_esp_to_eng(txt):
  return txt.replace('enero','jan').replace('febrero','feb').replace('marzo','mar').replace('abril','apr').replace('mayo','may').replace('junio','jun').replace('julio','jul').replace('agosto','aug').replace('septiembre','sep').replace('octubre','oct').replace('noviembre','nov').replace('diciembre','dec')

def mes_esp_to_eng_upper(txt):
  return txt.replace('Enero','jan').replace('Febrero','feb').replace('Marzo','mar').replace('Abril','apr').replace('Mayo','may').replace('Junio','jun').replace('Julio','jul').replace('Agosto','aug').replace('Septiembre','sep').replace('Octubre','oct').replace('Noviembre','nov').replace('Diciembre','dec')

def timpestamp_pag12(fecha):  
    return datetime.strptime(fecha, '%Y-%m-%d')
def timpestamp_clarin(fecha):  
    return datetime.strptime(fecha, '%d/%m/%Y %H:%M')
def timpestamp_lanacion(fecha):  
    return datetime.strptime(fecha, '%d de %b de %Y  •  %H:%M')
def timpestamp_elcronista(fecha):  
    return datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
def timpestamp_infobae(fecha):  
    return datetime.strptime(fecha, '%d de %b de %Y')
def timpestamp_ambito(fecha):  
    return datetime.strptime(fecha, '%d %b %Y - %H:%M')

In [12]:
# WARNING!: este código solo sirve en caso de estar ejecutando el scrap desde la pc y no desde colab
# ya que colab no permite abrir popup.
"""
 Sirve para los casos en los que las páginas de secciones de noticias vayan cargando noticias hacia abajo, 
 la función abre un popup no modal para condicionar la página principal de noticias, cargar todas las noticias que se quieran escrapear y
 por último se debe dar clic en el botón "SCRAP!" del popup no modal.
"""

import tkinter as tk
from tkinter import messagebox

root= tk.Tk()

def ExitApplication():
    root.destroy()

def display_buttom():
    canvas1 = tk.Canvas(root, width = 300, height = 100)
    canvas1.pack()
            
    button1 = tk.Button (root, text='SCRAP!',command=ExitApplication,bg='green',fg='white')
    canvas1.create_window(150, 50, window=button1)
    
    root.mainloop()

TclError: ignored

## Página 12



In [13]:
def obtiene_noticias_pag12(pag_base,pag):
  resp = requests.get(pag_base + str(pag))
  
  soup = BeautifulSoup(resp.content, features = "html")
  div_noticias = soup.findAll("div", {"class": "article-item__content"})
  noticias = []
  for div in div_noticias:
    noticias.append(div.find("a").get("href"))
  return noticias

In [14]:
def lee_noticia_pag12(link_noticia):
    print(link_noticia)
    n = {}

    n['link_noticia'] = link_noticia
    resp = requests.get(link_noticia)

    # si la respuesta del servidor es mala, sale de la ejecución de esta noticia.
    if resp.ok == False:
      return None

    soup = BeautifulSoup(resp.content, features = "html")

    # obtiene distintas partes de la noticia:
    articulo_completo = soup.find("div", {"class": "article-inner padding-right"})
    info = articulo_completo.find("div", {"class": "article-info"})
    titulos = articulo_completo.find("div", {"class": "article-titles"})
    body = articulo_completo.find("div", {"class": "article-body diario"})
    imagen = articulo_completo.find("div", {"class": "article-main-media-image"})


    # obtiene atributos

    try:
      raw_fecha = info.find("div",{"class": "time"}).find("span").attrs["datetime"]
      n['fecha'] = timpestamp_pag12(raw_fecha)
    except:
      n['fecha'] = None


    try:
      n['seccion'] = info.find("div", {"class": "suplement"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['seccion'] = None
    try:
      n['volanta'] = titulos.find("h2",{"class":"article-prefix"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['volanta'] = None
    try:
      n['titulo'] = titulos.find("h1",{"class":"article-title"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['titulo'] = None
    try:
      n['resumen'] = titulos.find("div",{"class":"article-summary"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['resumen'] = None
    try:
      n['epigrafe'] = imagen.find("div",{"class": "article-main-media-text"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['epigrafe'] = None

    articulo_texto = body.find("div",{"class":"article-text"})
    parrafos = articulo_texto.findAll("p")
    titulos_parrafos = articulo_texto.findAll("h2")

    text_parrafo = ""
    for item in parrafos:
      text_parrafo += "||" + item.text

    text_titulos_parrafos = ""
    for item in titulos_parrafos:
      text_titulos_parrafos += "||" + item.text

    n['titulos_parrafos'] = text_titulos_parrafos.strip().replace("\r","").replace("\n","").replace("\t","")
    n['parrafos'] = text_parrafo.strip().replace("\r","").replace("\n","").replace("\t","")

    return n
  

In [15]:
def scrap_pag12(desde_pag,hasta_pag):
  l = []
  for pag in range(desde_pag,hasta_pag + 1):
    noticias = obtiene_noticias_pag12("https://www.pagina12.com.ar/secciones/economia?page=",pag)
    for noticia in noticias:
      scrap_not = lee_noticia_ambito(noticia)
      if scrap_not != None:
        l.append(scrap_not)
  return pd.DataFrame(l)

In [16]:
df_pagina12 = scrap_pag12(0,0)

https://www.pagina12.com.ar/302429-auh-recomponer-derechos-a-los-ninos-es-fundamental
https://www.pagina12.com.ar/302417-kulfas-hay-senales-claras-de-recuperacion
https://www.pagina12.com.ar/302416-presupuesto-2021-claves-para-volver-a-crecer
https://www.pagina12.com.ar/302415-presupuesto-2021-los-numeros-del-proyecto
https://www.pagina12.com.ar/302396-mas-creditos-subsidiados-al-sector-productivo-y-mas-deposito
https://www.pagina12.com.ar/302385-vicentin-allanaron-las-oficinas-del-banco-nacion
https://www.pagina12.com.ar/302380-repunte-dispar-de-la-demanda-electrica
https://www.pagina12.com.ar/302372-fuerte-caida-del-dolar-contado-con-liquidacion
https://www.pagina12.com.ar/302371-un-aeropuerto-que-no-despega
https://www.pagina12.com.ar/302364-alberto-fernandez-con-inversion-se-va-a-generar-trabajo
https://www.pagina12.com.ar/302348-mercado-pago-aclaro-que-el-cobro-por-transferencias-no-es-un


In [17]:
df_pagina12.head()

,link_noticia,fecha,seccion,volanta,titulo,resumen,epigrafe,titulos_parrafos,parrafos
0,https://www.pagina12.com.ar/302429-auh-recompo...,2020-10-28,Economía,Expectativas por la ampliación de la asignació...,"AUH: ""Recomponer derechos a los niños es funda...",Favorables repercusiones a la ampliación de la...,Imagen: Pablo Piovano,,||El Gobierno anunciará este jueves la ampliac...
1,https://www.pagina12.com.ar/302417-kulfas-hay-...,2020-10-28,Economía,El ministro de Desarrollo Productivo indicó qu...,"Kulfas: ""Hay señales claras de recuperación""",None,Imagen: Leandro Teysseire,,"||El ministro de Desarrollo Productivo, Matías..."
2,https://www.pagina12.com.ar/302416-presupuesto...,2020-10-28,Economía,El proyecto de gastos y recursos del Estado na...,Presupuesto 2021: claves para volver a crecer,El 65 por ciento se destinará al pago de jubil...,El ministro Martín Guzmán estima un crecimient...,||Prioridades||Agregados y cambios,||El proyecto de Presupuesto 2021 contempla un...
3,https://www.pagina12.com.ar/302415-presupuesto...,2020-10-28,Economía,Contempla un crecimiento del 5 por ciento del ...,Presupuesto 2021: los números del proyecto,Las estimaciones que proyecta el Ministerio de...,Imagen: NA,,||La Cámara de Diputados de la Nación debate e...
4,https://www.pagina12.com.ar/302396-mas-credito...,2020-10-28,Economía,El Banco Central detalló cómo está funcionando...,Más créditos subsidiados al sector productivo ...,El Informe sobre Bancos del BCRA revela que el...,"En el acumulado de 8 meses del año, el sistema...",,||El Banco Central destacó este miércoles que ...


## Clarín

In [19]:
def obtiene_noticias_clarin(pag, usar_webdriver = False, webdriver_dir = ""):
  
  if usar_webdriver == False:
    resp = requests.get(pag)
    soup = BeautifulSoup(resp.content, features = "html")
  else:
    browser = webdriver.Chrome(webdriver_dir)
    browser.get(pag)
    display_buttom()
    soup = BeautifulSoup(browser.page_source,"html.parser",multi_valued_attributes=None)

  div_noticias = soup.findAll("article")
  noticias = []
  for div in div_noticias:
    try:
      noticias.append("https://www.clarin.com/" + div.find("a").get("href"))
    except:
      pass
  return noticias

In [20]:
def lee_noticia_clarin(link_noticia):
    print(link_noticia)
    n = {}
    n['link_noticia'] = link_noticia
    resp = requests.get(link_noticia)
    
    # si la respuesta del servidor es mala, sale de la ejecución de esta noticia.
    if resp.ok == False:
      return None

    soup = BeautifulSoup(resp.content, features = "html")

    # obtiene distintas partes de la noticia:
    try:
      titulos = soup.find("div", {"class": "title"})
    except:
      return {'link_noticia': link_noticia}
    try:
      imagen_ppal = soup.find("div",{"id": "galeria-trigger"})
    except:
      n['epigrafe'] = None
      imagen_ppal = None

    try:
      body = soup.find("div", {"class": "body-nota"})
    except:
      return {'link_noticia': link_noticia}

    # obtiene atributos

    try:
      raw_fecha = soup.find("span",{"class": "publishedDate"}).text.replace("\r","").replace("\n","").replace("\t","")
      n['fecha'] = timpestamp_clarin(raw_fecha)
    except:
      n['fecha'] = None

    try:
      n['seccion'] = soup.find("li",{"class":"section-name-content"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['seccion'] = None
    try:
      n['volanta'] = titulos.find("p",{"class":"volanta"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['volanta'] = None
    try:
      n['titulo'] = titulos.find("h1",{"id":"title"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['titulo'] = None
    try:
      n['resumen'] = titulos.find("div",{"class":"bajada"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['resumen'] = None

    if imagen_ppal != None:
      try:
        n['epigrafe'] = imagen_ppal.find("p").text.replace("\r","").replace("\n","").replace("\t","")
      except:
        n['epigrafe'] = None

    try:
      parrafos = body.findAll("p")
    except:
      return {'link_noticia': link_noticia, 'fecha': None, 'volanta': None, 'titulo': None, 'resumen': None, 'epigrafe': None, 'titulos_parrafos': None, 'parrafos': None,}
    titulos_parrafos = body.findAll("h2")

    text_parrafo = ""
    for item in parrafos:
      if "COMENTARIOS" in item.text:
        break
      else:
        text_parrafo += "||" + item.text

    text_titulos_parrafos = ""
    for item in titulos_parrafos:
       text_titulos_parrafos += "||" + item.text

    n['titulos_parrafos'] = text_titulos_parrafos.strip().replace("TEMAS QUE APARECEN EN ESTA NOTA","").replace("\r","").replace("\n","").replace("\t","")
    n['parrafos'] = text_parrafo.strip().replace("TEMAS QUE APARECEN EN ESTA NOTA","").replace("\r","").replace("\n","").replace("\t","")

    return n
  

In [21]:
def scrap_clarin(pag):
  l = []

  noticias = obtiene_noticias_clarin(pag)
  for noticia in noticias:
    scrap_not = lee_noticia_ambito(noticia)
    if scrap_not != None:
      l.append(scrap_not)
  return pd.DataFrame(l)

In [22]:
df_clarin = scrap_clarin("https://www.clarin.com/economia/")

https://www.clarin.com//economia/gobierno-autorizara-prepagas-subir-15-noviembre-10-enero_0_ik3TMHeXY.html
https://www.clarin.com//economia/gasto-equipamiento-hizo-trabajador-relacion-dependencia-puede-deducir-ganancias_0_6_gXx3y60.html
https://www.clarin.com//economia/cervecera-ccu-anuncio-inversiones-4500-millones-planta-lujan_0_ttez63Tyq.html
https://www.clarin.com//rural/espera-sector-agricola-ingrese-11-000-millones-dolares-proximos-5-meses_0_H_GlqLMEu.html
https://www.clarin.com//economia/dolar-cuesta-gobierno-bajar-brecha-cambiaria_0_twjuAjrj8.html
https://www.clarin.com//economia/atp-7-asistencia-salarial-solo-sectores-criticos_0_B90aW8cJY.html
https://www.clarin.com//economia/desploma-wall-street-castiga-acciones-argentinas-nueva-york_0_Ns16bC3i7.html
https://www.clarin.com//economia/mercado-pago-informo-transferir-dinero-terceros-comision_0_wBdUJHJUM.html
https://www.clarin.com//economia/dolar-blue-cae-tercer-dia-consecutivo-cotiza-debajo-180_0_Frup1PpNU.html
https://www.clar

In [23]:
df_clarin.head()

,link_noticia,fecha,seccion,volanta,titulo,resumen,epigrafe,titulos_parrafos,parrafos
0,https://www.clarin.com//economia/gobierno-auto...,2020-10-28 21:49:00,Economía,Impacto en el bolsillo,El Gobierno autorizará a las prepagas subir 15...,El último incremento fue en diciembre. Falta t...,Aumentos de cuotas en las prepagas. / Shutters...,||,"||Las prepagas aseguran que, por los increment..."
1,https://www.clarin.com//economia/gasto-equipam...,2020-10-28 20:09:00,Economía,Impuestos,El gasto en equipamiento que hizo un trabajado...,Es para el caso de gastos que hubieran corresp...,Home office,||ATP 7: la asistencia salarial será sólo para...,||Los gastos en que incurre el trabajador en r...
2,https://www.clarin.com//economia/cervecera-ccu...,2020-10-28 19:35:00,Economía,Mercado de bebidas,La cervecera CCU anunció inversiones de más de...,El proyecto permitirá duplicar la capacidad pr...,El Presidente Alberto Fernandez en la planta d...,||,||En un acto oficial del que participó el pres...
3,https://www.clarin.com//rural/espera-sector-ag...,2020-10-28 19:26:00,Rural,Agroindustria,Se espera que el sector agrícola ingrese 11.00...,Es por la liquidación de la soja pendiente de ...,Las exportaciones agrícolas representó el 51% ...,||Agricultura 4.0: tecnologías para entender y...,"||El Gobierno, urgido de dólares, mira cada ve..."
4,https://www.clarin.com//economia/dolar-cuesta-...,2020-10-28 18:39:00,Economía,Oficial vs Contado con liqui,Dólar: cuánto le cuesta al Gobierno bajar la b...,El Banco Central y la ANSeS se desprenden a pr...,Miguel Pesce y Martín Guzmán,||,"||Se insinuó tras el canje de deuda, cuando el..."


## La Nación

In [24]:
def obtiene_noticias_lanacion(pag, usar_webdriver = False, webdriver_dir = ""):
  
  if usar_webdriver == False:
    resp = requests.get(pag)
    soup = BeautifulSoup(resp.content, features = "html")
  else:
    browser = webdriver.Chrome(webdriver_dir)
    browser.get(pag)
    display_buttom()
    soup = BeautifulSoup(browser.page_source,"html.parser",multi_valued_attributes=None)

    
  div_noticias = soup.findAll("article", {"class": "nota"})
  noticias = []
  for div in div_noticias:
    noticias.append("https://www.lanacion.com.ar/" + div.find("h2").find("a").get("href"))
  return noticias

In [25]:
def lee_noticia_lanacion(link_noticia):
    print(link_noticia)
    n = {}
    n['link_noticia'] = link_noticia
    resp = requests.get(link_noticia)
    
    # si la respuesta del servidor es mala, sale de la ejecución de esta noticia.
    if resp.ok == False:
      return None

    soup = BeautifulSoup(resp.content, features = "html")

    # obtiene distintas partes de la noticia:
    try:
      articulo_completo = soup.find("article", {"id": "nota"})
    except:
      return {'link_noticia': link_noticia}
    # info = articulo_completo.find("div", {"class": "article-info"})
    try:
      titulos = articulo_completo.find("section", {"class": "encabezado"})
    except:
      return {'link_noticia': link_noticia}
    try:
      body = articulo_completo.find("section", {"id": "cuerpo"})
    except:
      return {'link_noticia': link_noticia}

    # obtiene atributos

    try:
      raw_fecha = body.find("section", {"class":"fecha"}).text.replace("\r","").replace("\n","").replace("\t","")
      n['fecha'] = timpestamp_lanacion(mes_esp_to_eng(raw_fecha))
    except:
      n['fecha'] = None

    try:
      n['seccion'] = titulos.find("strong", {"class": "categoria"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['seccion'] = None
    try:
      n['titulo'] = titulos.find("h1",{"class":"titulo"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['titulo'] = None
    try:
      n['resumen'] = titulos.find("div",{"class":"article-summary"}).text
    except:
      n['resumen'] = None
    try:
      n['epigrafe'] = body.find("epigrafe").text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['epigrafe'] = None


    parrafos = body.findAll("p")
    titulos_parrafos = body.findAll("h2")

    text_parrafo = ""
    for item in parrafos:
      text_parrafo += "||" + item.text

    text_titulos_parrafos = ""
    for item in titulos_parrafos:
      text_titulos_parrafos += "||" + item.text

    n['titulos_parrafos'] = text_titulos_parrafos.strip().replace("\r","").replace("\n","").replace("\t","")
    n['parrafos'] = text_parrafo.strip().replace("Conforme a los criterios de","").replace("\r","").replace("\n","").replace("\t","")

    return n

In [26]:
def scrap_lanacion(pags):
  l = []
  for pag in pags:
    noticias = obtiene_noticias_lanacion(pag)
    for noticia in noticias:
      scrap_not = lee_noticia_ambito(noticia)
      if scrap_not != None:
        l.append(scrap_not)
  df = pd.DataFrame(l)
  df['volanta'] = None
  return df

In [27]:
pags = ["https://www.lanacion.com.ar/economia/negocios",
        # "https://www.lanacion.com.ar/dolar-hoy",
        # "https://www.lanacion.com.ar/economia/campo",
        # "https://www.lanacion.com.ar/tema/emprendedores-tid53673"
        ]
df_lanacion = scrap_lanacion(pags)

https://www.lanacion.com.ar//economia/negocios/ccu-cerveceria-invierte-4500-millones-duplicar-produccion-nid2492490
https://www.lanacion.com.ar//economia/negocios/impuesto-al-cheque-se-aplicara-billeteras-digitales-nid2492452
https://www.lanacion.com.ar//economia/negocios/canje-por-obligacion-del-banco-central-irsa-nid2491545
https://www.lanacion.com.ar//economia/negocios/new-style-el-particular-caso-tienda-fundo-nid2490270
https://www.lanacion.com.ar//economia/negocios/pesimismo-empresario-respecto-economia-empleo-inversion-nid2490236
https://www.lanacion.com.ar//economia/negocios/buquebus-confirmo-invertira-us30-millones-luego-pandemia-nid2490404
https://www.lanacion.com.ar//economia/negocios/cristiano-rattazzi-no-entiendo-esta-negacion-ser-nid2490239
https://www.lanacion.com.ar//economia/negocios/dolar-sin-techo-alertan-impacto-gondolas-llegara-nid2488532
https://www.lanacion.com.ar//economia/negocios/economia-arte-van-mano-nid2484793
https://www.lanacion.com.ar//economia/negocios/f

In [28]:
df_lanacion.head()

,link_noticia,fecha,seccion,titulo,resumen,epigrafe,titulos_parrafos,parrafos,volanta
0,https://www.lanacion.com.ar//economia/negocios...,2020-10-28 15:12:00,Economía,CCU: la cervecera invertirá $4500 millones par...,None,"Fernando Sanchis (CCU), el gobernador de la pr...","||Correo Compras: zonas de entrega, cómo usarl...",||Esta mañana se llevó a cabo un acto en el qu...,None
1,https://www.lanacion.com.ar//economia/negocios...,2020-10-28 14:34:00,Economía,Mercado Pago y el impuesto al cheque: polémica...,None,"La nueva funcionalidad de Mercado Pago, que pe...",||Cómo abandonar la mentalidad de empleado||Pe...,"||""Hola. Te queríamos contar que pagarás impue...",None
2,https://www.lanacion.com.ar//economia/negocios...,2020-10-28 11:19:00,Economía,"Canje: por obligación del Banco Central, IRSA ...",None,Las empresas del grupo controlado por Eduardo ...,"||Correo Compras: zonas de entrega, cómo usarl...",||Primero fue el turno del Banco Hipotecario y...,None
3,https://www.lanacion.com.ar//economia/negocios...,2020-10-28 09:55:00,Economía,New Style: el particular caso de la tienda que...,None,Día de la inauguración del local de San Martín...,||Coronavirus hoy en la Argentina y el mundo: ...,||A fines de 1880 la Argentina era el quinto e...,None
4,https://www.lanacion.com.ar//economia/negocios...,2020-10-27 15:27:00,Economía,"Pesimismo empresario respecto de la economía, ...",None,Encuesta. La mayoría de los empresarios cree q...,||CCU: la cervecera invertirá $4500 millones p...,||Las expectativas de los empresarios argentin...,None


## El cronista

In [29]:
def obtiene_noticias_elcronista(pag, usar_webdriver = False, webdriver_dir = ""):
  
  if usar_webdriver == False:
    resp = requests.get(pag)
    soup = BeautifulSoup(resp.content, features = "html")
  else:
    browser = webdriver.Chrome(webdriver_dir)
    browser.get(pag)
    display_buttom()
    soup = BeautifulSoup(browser.page_source,"html.parser",multi_valued_attributes=None)

    
  div_noticias = soup.findAll("article")
  noticias = []
  for div in div_noticias:
    link = div.find("a").get("href")
    if link.startswith("http",0,4):
      pass
    else:
      noticias.append("https://www.cronista.com/" + link)
  return noticias

In [30]:
def lee_noticia_elcronista(link_noticia):
    print(link_noticia)
    n = {}
    n['link_noticia'] = link_noticia
    resp = requests.get(link_noticia)
    
    # si la respuesta del servidor es mala, sale de la ejecución de esta noticia.
    if resp.ok == False:
      return None

    soup = BeautifulSoup(resp.content, features = "html")

    # obtiene distintas partes de la noticia:

    try:
      titulos = soup.find("header", {"class": "header-nota"})
    except:
      return {'link_noticia': link_noticia}

    try:
      body = soup.find("div", {"class": "article-container"})
      # return body
    except:
      return {'link_noticia': link_noticia}

    # obtiene atributos

    try:
      raw_fecha = soup.find("meta", {"itemprop":"datePublished"}).attrs['content']
      n['fecha'] = timpestamp_elcronista(raw_fecha)
    except:
      n['fecha'] = None

    try:
      n['seccion'] = titulos.find("span",{"class":"entry-label-2"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['seccion'] = None
    try:
      n['titulo'] = titulos.find("h1",{"class":"titulo-nota"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['titulo'] = None
    try:
      n['resumen'] = titulos.find("p",{"class":"bajada"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['resumen'] = None
    try:
      n['epigrafe'] = soup.find("figcaption",{"class": "note-cape"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['epigrafe'] = None

    
    parrafos = body.findAll("p")

    contenido_listas = body.findAll("li")

    text_parrafo = ""
    for item in parrafos:
      text_parrafo += "||" + item.text

    for item in contenido_listas:
      text_parrafo += "||" + item.text

    titulos_parrafos = body.findAll("h4")
    text_titulos_parrafos = ""
    for item in titulos_parrafos:
      text_titulos_parrafos += "||" + item.text
    
    titulos_parrafos = body.findAll("h3")
    for item in titulos_parrafos:
      text_titulos_parrafos += "||" + item.text

    n['titulos_parrafos'] = text_titulos_parrafos.strip().replace("\r","").replace("\n","").replace("\t","")
    n['parrafos'] = text_parrafo.strip().replace("\r","").replace("\n","").replace("\t","")

    return n

In [31]:
def scrap_elcronista(pag):
  l = []

  noticias = obtiene_noticias_elcronista(pag)
  for noticia in noticias:
    scrap_not = lee_noticia_ambito(noticia)
    if scrap_not != None:
      l.append(scrap_not)
  df = pd.DataFrame(l)
  df['volanta'] = None
  return df

In [36]:
df_elcronista = scrap_elcronista("https://www.cronista.com/seccion/economia_politica/")

https://www.cronista.com//economiapolitica/EDEA-invierte-150-millones-para-consolidar-el-sistema-electrico-de-Mar-del-Plata-20201028-0053.html
https://www.cronista.com//economiapolitica/Presupuesto-2021-pagaran-deuda-con-petroleras-por--29.514-millones-por-la-devaluacion-2018-20201028-0048.html
https://www.cronista.com//economiapolitica/Maximo-Kirchner-se-reunio-con-referentes-del-sector-de-biocombustible-20201028-0035.html
https://www.cronista.com//economiapolitica/ATP-el-Gobierno-definio-un-fuerte-ajuste-en-la-asistencia-a-empresas-golpeadas-por-la-pandemia-20201028-0031.html
https://www.cronista.com//economiapolitica/ATP-7-que-empresas-pueden-pedir-el-subsidio-y-cuando-y-como-se-cobra-20201028-0019.html
https://www.cronista.com//economiapolitica/Rodriguez-Larreta-contra-las-tomas-Es-obligacion-del-Estado-proteger-la-propiedad-privada-20201028-0029.html
https://www.cronista.com//economiapolitica/Arranca-el-debate-en-Diputados-por-el-Presupuesto-2021-20201028-0025.html
https://www.cro

In [67]:
df_elcronista.tail()

,link_noticia,fecha,seccion,titulo,resumen,epigrafe,titulos_parrafos,parrafos,volanta
13,https://www.cronista.com//economiapolitica/Tra...,2020-10-27 10:18:05,Economía y política,"Tras la carta, pero sin la presencia de Cristi...",Encabezó la ceremonia para conmemorar los 10 a...,None,,||En un emotivo discurso que terminó con lágri...,None
14,https://www.cronista.com//economiapolitica/Dol...,2020-10-27 08:53:37,Economía y política,"Dólar, inflación, déficit fiscal y PBI: las ex...",Las previsiones del sector privado sobre las v...,"Dólar, inflación, déficit fiscal y PBI: las ex...",,||Los analistas y consultoras del sector priva...,None
15,https://www.cronista.com//economiapolitica/Que...,2020-10-27 06:36:56,Economía y política,"Quedan cubrir pagos de deuda por más de $ 1,04...",Noviembre y diciembre aparecían como los meses...,"Quedan cubrir pagos de deuda por más de $ 1,04...",,||El Gobierno celebró la colocación de más de ...,None
16,https://www.cronista.com//economiapolitica/Neu...,2020-10-27 05:57:17,Economía y política,Neuquén no paga intereses de un bono en dólare...,“La Provincia enfrenta desafíos económicos sin...,Neuquén no paga intereses de un bono en dólare...,,"||Neuquén anunció que no pagará los u$s 13,7 m...",None
17,https://www.cronista.com//economiapolitica/Bon...,2020-10-27 03:40:47,Economía y política,Bono atado al dólar: Economía captó el equival...,"Con la subasta de cinco títulos, el Tesoro con...",Bono atado al dólar: Economía captó el equival...,,||Luego de captar más de $ 254.000 millones en...,None


## Infobae

In [43]:
def obtiene_noticias_infobae(pag, usar_webdriver = False, webdriver_dir = ""):
  
  if usar_webdriver == False:
    resp = requests.get(pag)
    soup = BeautifulSoup(resp.content, features = "html")
  else:
    browser = webdriver.Chrome(webdriver_dir)
    browser.get(pag)
    display_buttom()
    soup = BeautifulSoup(browser.page_source,"html.parser",multi_valued_attributes=None)

  noticias = []

  # obtiene noticia principal
  main_new = soup.find("div",{"class": "main_new col mobile-12 tablet-8"})
  noticias.append("https://www.infobae.com/" + main_new.find("a").get("href"))

  # obtiene noticias secundarias
  second_new_panel = soup.find("div",{"class": "secondary_news col mobile-12 tablet-4"})
  second_news = second_new_panel.findAll("a")
  for link in second_news:
    noticias.append("https://www.infobae.com/" + link.get("href"))

  # obtiene noticias del body superior
  body_sup = soup.find("div",{"class": "body | page-container margin_auto grid width_full"})
  body_noticias = body_sup.findAll("div",{"class": "width_full padding_vertical_10px "})
  for div in body_noticias:
    links = div.findAll("a")
    for link in links:
      noticias.append("https://www.infobae.com/" + link.get("href"))
  
  # obtiene las noticias del body inferior:
  body_inf = soup.find("div",{"class": "minibody | inner_minibody grid col widescreen-8 desktop-8 tablet-8"})
  body_noticias_encabezados = body_inf.findAll("h2")
  for noticia in body_noticias_encabezados:
    noticias.append("https://www.infobae.com/" + noticia.find("a").get("href"))

  # Obtiene las noticias del panel dolar
  dolar_panel = soup.find("div",{"id": "0fzxTVPrCWPY5MC"})
  dolar_noticias = dolar_panel.findAll("div",{"class": "hidden_desktop"})
  for noticia in dolar_noticias:
    noticias.append("https://www.infobae.com/" + noticia.find("a").get("href"))
  return list(set(noticias))

In [44]:
def lee_noticia_infobae(link_noticia):
    print(link_noticia)
    n = {}
    n['link_noticia'] = link_noticia
    resp = requests.get(link_noticia)

    # si la respuesta del servidor es mala, sale de la ejecución de esta noticia.
    if resp.ok == False:
      return None

    soup = BeautifulSoup(resp.content, features = "html")

    # obtiene distintas partes de la noticia:

    try:
      titulos = soup.find("div", {"class": "article-header font_tertiary"})
    except:
      return {'link_noticia': link_noticia}

    # obtiene atributos

    try:
      raw_fecha = soup.find("div", {"class":"datetime | display_inline byline_datetime"}).text.replace("\r","").replace("\n","").replace("\t","")
      n['fecha'] = timpestamp_infobae(mes_esp_to_eng_upper(raw_fecha))
    except:
      n['fecha'] = None

    try:
      n['seccion'] = titulos.find("div").text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['seccion'] = None
    try:
      n['titulo'] = titulos.find("h1").text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['titulo'] = None
    try:
      n['resumen'] = titulos.find("h2").text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['resumen'] = None
    try:
      n['epigrafe'] = soup.find("figcaption",{"class": "padding_vertical_15px padding_horizontal_15px"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['epigrafe'] = None


    parrafos = soup.findAll("p",{"class": "font_tertiary paragraph"})

    text_parrafo = ""
    for item in parrafos:
      text_parrafo += "||" + item.text

    # for item in contenido_listas:
    #   text_parrafo += ". " + item.text

    # titulos_parrafos = body.findAll("h4")
    # text_titulos_parrafos = ""
    # for item in titulos_parrafos:
    #   text_titulos_parrafos += " " + item.text
    
    # titulos_parrafos = body.findAll("h3")
    # for item in titulos_parrafos:
    #   text_titulos_parrafos += " " + item.text

    # n['titulos_parrafos'] = text_titulos_parrafos.strip().replace("\r","").replace("\n","").replace("\t","")
    n['parrafos'] = text_parrafo.strip().replace("\r","").replace("\n","").replace("\t","")

    return n

In [45]:
def scrap_infobae(pag):
  l = []

  noticias = obtiene_noticias_infobae(pag)
  for noticia in noticias:
    scrap_not = lee_noticia_ambito(noticia)
    if scrap_not != None:
      l.append(scrap_not)
  df = pd.DataFrame(l)
  df['volanta'] = None
  df['titulos_parrafos'] = None
  return df

In [46]:
df_infobae = scrap_infobae("https://www.infobae.com/economia/")

https://www.infobae.com//economia/2020/10/28/boom-de-compras-a-dolar-oficial-el-free-shop-de-ezeiza-el-ultimo-lugar-para-pagar-a-84-y-sin-impuesto-solidario/
https://www.infobae.com//economia/2020/10/28/ocho-de-cada-diez-personas-estan-dispuestas-a-aceptar-una-oferta-laboral-para-irse-del-pais/
https://www.infobae.com//economia/2020/10/28/companeros-si-hay-que-mudarse-mudense-la-orden-a-los-diputados-oficialistas-para-que-no-les-falle-la-conexion-a-internet-para-votar-el-presupuesto/
https://www.infobae.com//economia/2020/10/28/cual-es-el-costo-al-que-se-endeudo-el-estado-con-el-bono-dollar-linked-que-emitio-martin-guzman/
https://www.infobae.com//economia/2020/10/28/atp-7-restringen-el-subsidio-estatal-para-pagar-parte-de-los-sueldos-que-solo-se-mantendra-para-los-sectores-criticos/
https://www.infobae.com//economia/networking/2020/10/20/premiaron-a-telecom-por-su-rapida-implementacion-de-trabajo-remoto-en-el-comienzo-de-la-pandemia/
https://www.infobae.com//economia/networking/2020/1

In [47]:
df_infobae

,link_noticia,fecha,seccion,titulo,resumen,epigrafe,parrafos,volanta,titulos_parrafos
0,https://www.infobae.com//economia/2020/10/28/b...,2020-10-28,Economía,Boom de compras a dólar oficial: el free shop ...,El Duty Free Shop del Aeropuerto Internacional...,Se puede acceder a productos importados sin el...,"||En estos días, los pasajeros que tienen la p...",None,None
1,https://www.infobae.com//economia/2020/10/28/o...,2020-10-28,Economía,Ocho de cada diez personas están dispuestas a ...,El dato surge de una encuesta realizada por el...,"Por la pandemia y la crisis económica, 8 de ca...",||El 82% de los encuestados por el Centro de O...,None,None
2,https://www.infobae.com//economia/2020/10/28/c...,2020-10-28,Economía,"“Compañeros, si hay que mudarse, múdense”: la ...",Tras una votación previa en la que hubo compli...,La virtualidad generó problemas para votar,||La Cámara de Diputados votó afirmativamente ...,None,None
3,https://www.infobae.com//economia/2020/10/28/c...,2020-10-28,Economía,Cuál es el costo al que se endeudó el Estado c...,"Para los analistas, la colocación de ayer del ...",La licitación de Martín Guzmán captó $119.473 ...,||Forzado a dar alguna clase de señal de limit...,None,None
4,https://www.infobae.com//economia/2020/10/28/a...,2020-10-28,Economía,ATP 7: restringen el subsidio estatal para pag...,Se reunió esta mañana en la Casa Rosada el gab...,El gabinete económico se reunió esta mañana en...,"||Como viene sucediendo todos los meses, la sé...",None,None
5,https://www.infobae.com//economia/networking/2...,2020-10-20,Networking,Premiaron a Telecom por su rápida implementaci...,La distinción fue otorgada por la organización...,La empresa de telecomunicaciones desarrolló su...,"||En medio de la pandemia, Telecom Argentina f...",None,None
6,https://www.infobae.com//economia/networking/2...,2020-10-22,Networking,Techint distinguió a más de 1.250 alumnos con ...,"Paolo Rocca, CEO del grupo, reconoció a estudi...","Paolo Rocca, presidente de Organización Techint",||La Organización Techint distinguió a más de ...,None,None
7,https://www.infobae.com//opinion/2020/10/28/ar...,2020-10-28,Opinión,"Argentina, zona liberada",Estamos en un punto de no retorno: o recuperam...,El presidente Alberto Fernández saluda al mini...,||Es unánime en toda la sociedad el reclamo al...,None,None
8,https://www.infobae.com//opinion/2020/10/28/la...,2020-10-28,Opinión,Las consecuencias de repetir errores,Una recapitulación de lo que produjo el populi...,"El presidente Fernández, días atrás en Neuquén...",||A la luz de los resultados que arrojó la ges...,None,None
9,https://www.infobae.com//economia/2020/10/28/l...,2020-10-28,Economía,Las reservas del Banco Central siguen en caída...,Los activos del BCRA restaron casi USD 250 mil...,El dólar libre acumula una baja de casi 9% en ...,||La caída de las cotizaciones del dólar alter...,None,None


# Ambito

In [48]:
def obtiene_noticias_ambito(pag_base, pag):
  noticias = []
  resp = requests.get(pag_base + str(pag))
  soup = BeautifulSoup(resp.content, features = "html")
  
  # obtenemos la noticia principal
  if pag == 0:
    main_new = soup.find("article",{"class": "highlighted-note-overprinted-title"})
    noticias.append(main_new.find("a").get("href"))
    main_new_2 = soup.find("article",{"class": "simple-note-square-image"})
    noticias.append(main_new_2.find("a").get("href"))
    four_news_panel = soup.find("section",{"class": "simple-list-square-image four-column"})
    four_news = four_news_panel.findAll("h2")
    for noticia in four_news:
      noticias.append(noticia.find("a").get("href"))

  # obtenemos las noticias listadas abajo
  div_noticias_list = soup.find("div", {"class": "list-articles"})
  div_noticias = div_noticias_list.findAll("h2", {"class": "title"})
  for div in div_noticias:
    noticias.append(div.find("a").get("href"))
  return noticias

In [70]:
def lee_noticia_ambito(link_noticia):
    print(link_noticia)
    n = {}
    n['link_noticia'] = link_noticia
    resp = requests.get(link_noticia)

    # si la respuesta del servidor es mala, sale de la ejecución de esta noticia.
    if resp.ok == False:
      return None

    soup = BeautifulSoup(resp.content, features = "html")

    # obtiene distintas partes de la noticia:

    try:
      titulos = soup.find("div", {"class": "detail-header"})
    except:
      return {'link_noticia': link_noticia, 'fecha': None, 'epigrafe': None, 'seccion': None, 'titulo': None, 'resumen': None, 'parrafos': None}

    # try:
    #   body = soup.find("div", {"class": "article-container"})
    # except:
    #   return {'link_noticia': link_noticia, 'fecha': = None, 'epigrafe': = None, 'seccion': = None, 'titulo': = None, 'resumen': = None, 'parrafos': None}

    # obtiene atributos
    try:
      raw_fecha = titulos.find("time").attrs['datetime'].replace("\r","").replace("\n","").replace("\t","")
      n['fecha'] = timpestamp_ambito(mes_esp_to_eng(raw_fecha))

    except:
      n['fecha'] = None

    try:
      n['seccion'] = titulos.find("span", {"class": "article-theme"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['seccion'] = None
    try:
      n['titulo'] = titulos.find("h1",{"class": "title"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['titulo'] = None
    try:
      n['resumen'] = titulos.find("h2", {"class": "excerpt"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['resumen'] = None
    try:
      n['epigrafe'] = soup.find("span",{"class": "epiinterior"}).text.replace("\r","").replace("\n","").replace("\t","")
    except:
      n['epigrafe'] = None

    text_parrafo = ""
    # Obtenemos info del primer parrafo:
    try:
      primer_parrafo = soup.find("section", {"class": "detail-body first-paragraph"})
      primeros_parrafos = primer_parrafo.findAll("p")
      for item in primeros_parrafos:
        text_parrafo += "||" + item.text
    except:
      pass

    # Obtenemos info de segundo parrafo
    try:
      segundo_parrafo = soup.find("section", {"class": "detail-body second-paragraph"})
      segundos_parrafos = segundo_parrafo.findAll("p")
      for item in segundos_parrafos:
        text_parrafo += "||" + item.text
    except:
      pass

    # Obtenemos los parrafos del body
    try:
      body = soup.find(lambda tag: tag.name == 'section' and 
                                   tag.get('class') == ['detail-body'])
      body_parrafos = body.findAll("p")
      for item in body_parrafos:
        text_parrafo += "||" + item.text
    except:
      pass

    # for item in contenido_listas:
    #   text_parrafo += ". " + item.text

    # titulos_parrafos = body.findAll("h4")
    # text_titulos_parrafos = ""
    # for item in titulos_parrafos:
    #   text_titulos_parrafos += " " + item.text
    
    # titulos_parrafos = body.findAll("h3")
    # for item in titulos_parrafos:
    #   text_titulos_parrafos += " " + item.text

    # n['titulos_parrafos'] = text_titulos_parrafos.strip().replace("\r","").replace("\n","").replace("\t","")
    n['parrafos'] = text_parrafo.strip().replace("El contenido al que quiere acceder es exclusivo para suscriptores.","").replace("\r","").replace("\n","").replace("\t","")

    return n

In [71]:
def scrap_ambito(desde_pag, hasta_pag):
  l = []
  for pag in range(desde_pag,hasta_pag + 1):
    noticias = obtiene_noticias_ambito("https://www.ambito.com/contenidos/economia.html/", pag)
    for noticia in noticias:
      scrap_not = lee_noticia_ambito(noticia)
      if scrap_not != None:
        l.append(scrap_not)
  df = pd.DataFrame(l)
  df['volanta'] = None
  df['titulos_parrafos'] = None
  return df

In [72]:
df_ambito = scrap_ambito(0,2)

https://www.ambito.com/economia/cnv/aumenta-control-operaciones-dolares-bursatiles-n5143996
https://www.ambito.com/economia/cnv/aumenta-control-operaciones-dolares-bursatiles-n5143996
https://www.ambito.com/economia/fmi/sergio-chodos-no-hay-razones-subyacentes-que-justifiquen-una-devaluacion-n5143972
https://www.ambito.com/economia/fmi/sergio-chodos-no-hay-razones-subyacentes-que-justifiquen-una-devaluacion-n5143972
https://www.ambito.com/economia/creditos/bancos-ya-entregaron-tasa-subsidiada-mas-600000-millones-n5143931
https://www.ambito.com/economia/creditos/bancos-ya-entregaron-tasa-subsidiada-mas-600000-millones-n5143931
https://www.ambito.com/economia/anses/cuando-cobro-quienes-depositan-jubilaciones-y-asignaciones-este-jueves-29-octubre-n5143914
https://www.ambito.com/economia/anses/cuando-cobro-quienes-depositan-jubilaciones-y-asignaciones-este-jueves-29-octubre-n5143914
https://www.ambito.com/economia/atp/7-abren-inscripcion-empresas-salarios-octubre-n5143912
https://www.ambit

# Crea el dataframe final

In [65]:
df_final = pd.concat([df_pagina12, 
                      df_clarin,
                      df_lanacion,
                      df_elcronista,
                      df_infobae,
                      df_ambito])

In [ ]:
today = str(datetime.date(datetime.now()))
df_final.to_csv(OUTPUT_DIR + "\\" + "_scrap_diarios_" +today.replace("-","") + ".csv",encoding="utf-8")    